In [ ]:
%matplotlib inline 

import six
import time
import Quandl
import calendar
import warnings
import numpy as np
import pandas as pd
from math import sqrt
from datetime import datetime

from sklearn import metrics
from sklearn.metrics import mean_squared_error as MSE

from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure import TanhLayer, SigmoidLayer, SoftmaxLayer
from pybrain.datasets import SupervisedDataSet, ClassificationDataSet

import sys
from os import listdir
from os.path import isfile, join
from helpers import features_analysis, procces_stocks, data_manipulation, download_quandl_data, ml_dataset, classifier_utils, report_generator, Iteration, Stacking, Boosting

warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
GOLD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/GOLD.csv')
SILVER = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/SILVER.csv')
PLAT = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/PLAT.csv')
OIL_BRENT = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/OIL_BRENT.csv')

USD_GBP = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/USD_GBP.csv')
JPY_USD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/JPY_USD.csv')
AUD_USD = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/AUD_USD.csv')

INDEX_DJIA = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_DJIA.csv')
INDEX_HSI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_HSI.csv')
INDEX_IBEX = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_IBEX.csv')
INDEX_N225 = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_N225.csv')
INDEX_SP500 = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_SP500.csv')
INDEX_AXJO = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_AXJO.csv')
INDEX_FCHI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_FCHI.csv')
INDEX_GDAXI = data_manipulation.read_csv_data('/Users/Pablo/Desktop/TFM/Data/INDEX_GDAXI.csv')

values_names = ['GOLD', 'SILVER', 'PLAT', 'OIL_BRENT', 'USD_GBP', 'JPY_USD', 'AUD_USD', 'DJIA', 'HSI', 'IBEX', 'N225', 'SP500', 'AXJO', 'FCHI', 'GDAXI']
values_dfs = [GOLD, SILVER, PLAT, OIL_BRENT, USD_GBP, JPY_USD, AUD_USD, INDEX_DJIA, INDEX_HSI, INDEX_IBEX, INDEX_N225, INDEX_SP500, INDEX_AXJO, INDEX_FCHI, INDEX_GDAXI]
values_cols = ['USD', 'Value', 'Open', 'Close', 'High', 'Low', 'Volume']
dict_dfs_cols = {}
dict_dfs_cols2 = {}

for index in range(len(values_names)):
    name = values_names[index]
    df = values_dfs[index]    
    cols = df.columns.values

    new_cols = [x for x in cols if x not in ['Date', 'USD', 'Value', 'Open', 'Close', 'High', 'Low', 'Volume']]    
    new_cols2 = [x for x in cols if x not in ['Date']]    

    dict_dfs_cols[name] = new_cols
    dict_dfs_cols2[name] = new_cols2

dataset = ml_dataset.generate_df_dataset(values_names, values_dfs, dict_dfs_cols)
dataset_all = ml_dataset.generate_df_dataset(values_names, values_dfs, dict_dfs_cols2)

#First 30 row
dataset = dataset[31:]
dataset = dataset.reset_index(drop=True)

dataset_all = dataset_all[31:]
dataset_all = dataset_all.reset_index(drop=True)

In [ ]:
SEED = 42
colY = 'IBEX_RD_B3_Close'
colsDiff = 1
colsToShift = 1

df_x = dataset.filter(regex=('RD_B3_Close'))
last_row = list(range(df_x.shape[0] - colsToShift, df_x.shape[0] ))
df_x = df_x.drop(last_row, axis=0)
df_x = df_x.drop(colY, axis=1)
        
df_y = dataset[colY].shift(-colsToShift)
last_row = list(range(df_y.shape[0] - colsToShift, df_y.shape[0] ))
df_y = df_y.drop(last_row, axis=0)

print 'Dataset shape %s' % str(dataset.shape)
print 'X shape %s' % str(df_x.shape)
print 'Y shape %s' % str(df_y.shape)

training_dates = Iteration.Iteration('2008-06-17', '2011-09-01')
testing_dates  = Iteration.Iteration('2012-09-04', '2014-10-06')
training_dates.calculate_indices(dataset)
testing_dates.calculate_indices(dataset)

trainDates = []
testDates = []
trainDates.append(training_dates.lowerIndex)
trainDates.append(training_dates.upperIndex)
testDates.append(testing_dates.lowerIndex)
testDates.append(testing_dates.upperIndex)

total = (trainDates[1]-trainDates[0]) + (testDates[1]-testDates[0])
tr = float(trainDates[1]-trainDates[0]) / total * 100.0
te = float(testDates[1]-testDates[0]) / total * 100.0

trainX, trainY, testX, testY = ml_dataset.train_arrays_experiments(df_x, df_y, trainDates, testDates)
trainY = trainY.reshape( -1, 1 )  
testY = testY.reshape( -1, 1 ) 

In [ ]:
ds = ClassificationDataSet(trainX.shape[1], nb_classes=2, class_labels=['Up','Down'])
ds.setField( 'input', trainX )
ds.setField( 'target', trainY )
#One hot encoding -> DS._convertToOneOfMany()
    
for hidden_neurons in [10]:

    input_size = ds.indim
    hidden_size = hidden_neurons
    target_size = ds.outdim
    validation_proportion = 0.15
    epochs = 1000
    continue_epochs = 10

    print "Building network..."
    net = buildNetwork(input_size, hidden_size, target_size, bias = True, hiddenclass=TanhLayer, outclass=SigmoidLayer)
    #SoftmaxLayer One Hot Encoding
    print "Training network..."
    trainer = BackpropTrainer( net, ds )
    train_mse, validation_mse = trainer.trainUntilConvergence( verbose = False, validationProportion = validation_proportion, 
                                                              maxEpochs = epochs, continueEpochs = continue_epochs )


    # Predicions
    ds_test = ClassificationDataSet( testX.shape[1], testY.shape[1])
    ds_test.setField( 'input', testX )
    ds_test.setField( 'target', testY )

    # predict
    preds = net.activateOnDataset( ds_test)

    mse = MSE( testY, preds )
    rmse = sqrt( mse )

    print "testing RMSE:", rmse
    mean_preds = np.round_(preds, decimals=0)
    print "score: ", metrics.accuracy_score(testY, mean_preds)